In [1]:
import optuna
import numpy as np
import pandas as pd
from optuna.integration.lightgbm import LightGBMTunerCV as lgbcv
from lightgbm import LGBMClassifier
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation, Dataset
from optuna.integration import LightGBMPruningCallback
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('data/train_cleaned.csv')
test = pd.read_csv('data/test_cleaned.csv')

In [3]:
train.dtypes

id                 int64
N_Days             int64
Drug               int64
Age                int64
Sex                int64
Ascites            int64
Hepatomegaly       int64
Spiders            int64
Edema             object
Bilirubin        float64
Cholesterol      float64
Albumin          float64
Copper           float64
Alk_Phos         float64
SGOT             float64
Tryglicerides    float64
Platelets        float64
Prothrombin      float64
Stage              int64
Status            object
dtype: object

In [4]:
# drop id
train = train.drop(['id'], axis=1)

X_train, X_val, y_train, y_val = train_test_split(train.drop(['Status'], axis=1), train['Status'], test_size=0.2, random_state=66)

In [5]:
# get list of categorical features
categorical_features = [col for col in X_train.columns if X_train[col].dtype == 'object']
# convert to category
for col in categorical_features:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')
categorical_features

['Edema']

In [6]:
# label encode target
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

# check label encoding, 0, 1, 2
le.classes_

array(['C', 'CL', 'D'], dtype=object)

In [7]:
# check target balance
np.unique(y_train, return_counts=True)

(array([0, 1, 2]), array([3962,  226, 2136], dtype=int64))

## LightGBM

### Auto Tuning

In [9]:
# auto tuner
auto_tuner_params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting': 'gbdt',
    'verbosity': -1,
    'num_class': 3,
    'auto_class_weights': 'balanced'
    }

train_data = Dataset(X_train, label=y_train, categorical_feature=categorical_features)

study_auto_lgb = optuna.create_study(direction='minimize', study_name='LightGBM Auto Tuner CV')

auto_tuner = lgbcv(
    params=auto_tuner_params,
    train_set=train_data,
    study=study_auto_lgb,
    callbacks=[early_stopping(2)],
    nfold=10,
    optuna_seed=666,
    seed=666
    )

In [10]:
auto_tuner.run()

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 2 rounds


feature_fraction, val_score: 0.458089:  14%|#4        | 1/7 [00:00<00:05,  1.06it/s]

Early stopping, best iteration is:
[39]	cv_agg's valid multi_logloss: 0.458089 + 0.0140713
Training until validation scores don't improve for 2 rounds


feature_fraction, val_score: 0.458089:  29%|##8       | 2/7 [00:01<00:04,  1.09it/s]

...

Early stopping, best iteration is:
[61]	cv_agg's valid multi_logloss: 0.446249 + 0.0171538
Training until validation scores don't improve for 2 rounds


min_child_samples, val_score: 0.442043:  80%|########  | 4/5 [00:05<00:01,  1.27s/it]

Early stopping, best iteration is:
[67]	cv_agg's valid multi_logloss: 0.444129 + 0.0169873
Training until validation scores don't improve for 2 rounds


min_child_samples, val_score: 0.442043: 100%|##########| 5/5 [00:06<00:00,  1.31s/it]

Early stopping, best iteration is:
[65]	cv_agg's valid multi_logloss: 0.442684 + 0.0172853


In [11]:
# fit model with best params
best_params = auto_tuner.best_params

best_model = LGBMClassifier(**best_params, random_state=666)
best_model.fit(X_train, y_train, categorical_feature=categorical_features)


LGBMClassifier(auto_class_weights='balanced', bagging_fraction=1.0,
               bagging_freq=0, boosting='gbdt', feature_fraction=0.4,
               feature_pre_filter=False, lambda_l1=1.9244268848741548,
               lambda_l2=2.5370679601187212e-08, metric='multi_logloss',
               num_class=3, objective='multiclass', random_state=666,
               verbosity=-1)

In [12]:
# predict on validation set with probabilities
y_prob = best_model.predict_proba(X_val)

# evaluate log_loss
from sklearn.metrics import log_loss
print(f'Train Log Loss: {log_loss(y_train, best_model.predict_proba(X_train))}')
print(f'Val Log Loss: {log_loss(y_val, y_prob)}')

Train Log Loss: 0.22732762814688576
Val Log Loss: 0.4172626803280405


### Manual Tuning

In [22]:
# manual define the params
def objective(trial):
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'boosting': trial.suggest_categorical('boosting', ['gbdt', 'dart','rf']),
        'verbosity': -1,
        'num_class': 3,
        'auto_class_weights': trial.suggest_categorical('auto_class_weights', ['balanced', 'sqrt_balanced', 'None']),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 2, 256),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': 666
        }
    train_data = Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    pruning_callback = LightGBMPruningCallback(trial, 'multi_logloss')
    cv_results = lgb.cv(
        params=params,
        train_set=train_data,
        num_boost_round = trial.suggest_int('num_boost_round', 100, 1000),
        nfold=10,
        seed=666,
        stratified=True,
        callbacks=[pruning_callback,early_stopping(10)],
        )
    return cv_results['valid multi_logloss-mean'][-1]


In [23]:
study = optuna.create_study(direction='minimize', study_name='LightGBM Manual Tuner CV', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=150)

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[624]	cv_agg's valid multi_logloss: 0.462485 + 0.0178214
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[568]	cv_agg's valid multi_logloss: 0.473098 + 0.0162952
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[7]	cv_agg's valid multi_logloss: 0.520291 + 0.0117702
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	cv_agg's valid multi_logloss: 0.52903 + 0.0134133
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[53]	cv_agg's valid multi_logloss: 0.502419 + 0.0129589
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[47]	cv_agg's valid multi_logloss: 0.503501 + 0.0167077
T

In [24]:
study.best_params

{'boosting': 'dart',
 'auto_class_weights': 'balanced',
 'learning_rate': 0.08771199438208843,
 'num_leaves': 42,
 'max_depth': 19,
 'min_data_in_leaf': 194,
 'feature_fraction': 0.6351623364144221,
 'bagging_fraction': 0.7001922449576534,
 'bagging_freq': 6,
 'lambda_l1': 0.14745561326671539,
 'lambda_l2': 0.034344251982640954,
 'min_child_samples': 14,
 'num_boost_round': 499}

In [25]:
# fit model with best params
best_params = study.best_params

best_model = LGBMClassifier(**best_params, random_state=666)
best_model.fit(X_train, y_train, categorical_feature=categorical_features)

[LightGBM] [Warning] Unknown parameter: auto_class_weights
[LightGBM] [Warning] min_data_in_leaf is set=194, min_child_samples=14 will be ignored. Current value: min_data_in_leaf=194
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.6351623364144221, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6351623364144221
[LightGBM] [Warning] lambda_l1 is set=0.14745561326671539, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.14745561326671539
[LightGBM] [Warning] lambda_l2 is set=0.034344251982640954, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.034344251982640954
[LightGBM] [Warning] bagging_fraction is set=0.7001922449576534, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7001922449576534
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] num_it

LGBMClassifier(auto_class_weights='balanced',
               bagging_fraction=0.7001922449576534, bagging_freq=6,
               boosting='dart', feature_fraction=0.6351623364144221,
               lambda_l1=0.14745561326671539, lambda_l2=0.034344251982640954,
               learning_rate=0.08771199438208843, max_depth=19,
               min_child_samples=14, min_data_in_leaf=194, num_boost_round=499,
               num_leaves=42, random_state=666)

In [26]:
# predict on validation set with probabilities
y_prob = best_model.predict_proba(X_val)

# evaluate log_loss
from sklearn.metrics import log_loss
print(f'Train Log Loss: {log_loss(y_train, best_model.predict_proba(X_train))}')
print(f'Val Log Loss: {log_loss(y_val, y_prob)}')

[LightGBM] [Warning] Unknown parameter: auto_class_weights
[LightGBM] [Warning] min_data_in_leaf is set=194, min_child_samples=14 will be ignored. Current value: min_data_in_leaf=194
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.6351623364144221, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6351623364144221
[LightGBM] [Warning] lambda_l1 is set=0.14745561326671539, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.14745561326671539
[LightGBM] [Warning] lambda_l2 is set=0.034344251982640954, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.034344251982640954
[LightGBM] [Warning] bagging_fraction is set=0.7001922449576534, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7001922449576534
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] Unknow

### Manual Tuning Round 2

In [27]:
# manual define the params
def objective(trial):
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'boosting': trial.suggest_categorical('boosting', ['gbdt', 'dart','rf']),
        'verbosity': -1,
        'num_class': 3,
        'auto_class_weights': trial.suggest_categorical('auto_class_weights', ['balanced', 'sqrt_balanced', 'None']),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'max_depth': trial.suggest_int('max_depth', 2, 100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 200),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': 666
        }
    train_data = Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    pruning_callback = LightGBMPruningCallback(trial, 'multi_logloss')
    cv_results = lgb.cv(
        params=params,
        train_set=train_data,
        num_boost_round = trial.suggest_int('num_boost_round', 100, 1000),
        nfold=10,
        seed=666,
        stratified=True,
        callbacks=[pruning_callback,early_stopping(10)],
        )
    return cv_results['valid multi_logloss-mean'][-1]

In [28]:
study = optuna.create_study(direction='minimize', study_name='LightGBM Manual Tuner CV', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=150)

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[12]	cv_agg's valid multi_logloss: 0.53568 + 0.013415
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	cv_agg's valid multi_logloss: 0.521948 + 0.012994
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[61]	cv_agg's valid multi_logloss: 0.522388 + 0.0141725
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[433]	cv_agg's valid multi_logloss: 0.528315 + 0.00844257
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[52]	cv_agg's valid multi_logloss: 0.500706 + 0.0134125
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	cv_agg's valid multi_logloss: 0.519135 + 0.0142205
Training until v

In [29]:
study.best_params

{'boosting': 'rf',
 'auto_class_weights': 'balanced',
 'learning_rate': 0.021519711417098856,
 'num_leaves': 78,
 'max_depth': 95,
 'min_data_in_leaf': 8,
 'feature_fraction': 0.6440892430178381,
 'bagging_fraction': 0.9408297820132444,
 'bagging_freq': 2,
 'lambda_l1': 3.3353038452842824e-06,
 'lambda_l2': 0.48998876620247134,
 'min_child_samples': 7,
 'num_boost_round': 422}

In [30]:
# fit model with best params
best_params = study.best_params

best_model = LGBMClassifier(**best_params, random_state=666)
best_model.fit(X_train, y_train, categorical_feature=categorical_features)

[LightGBM] [Warning] Unknown parameter: auto_class_weights
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=7 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] feature_fraction is set=0.6440892430178381, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6440892430178381
[LightGBM] [Warning] lambda_l1 is set=3.3353038452842824e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.3353038452842824e-06
[LightGBM] [Warning] lambda_l2 is set=0.48998876620247134, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.48998876620247134
[LightGBM] [Warning] bagging_fraction is set=0.9408297820132444, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9408297820132444
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] num_iterati

LGBMClassifier(auto_class_weights='balanced',
               bagging_fraction=0.9408297820132444, bagging_freq=2,
               boosting='rf', feature_fraction=0.6440892430178381,
               lambda_l1=3.3353038452842824e-06, lambda_l2=0.48998876620247134,
               learning_rate=0.021519711417098856, max_depth=95,
               min_child_samples=7, min_data_in_leaf=8, num_boost_round=422,
               num_leaves=78, random_state=666)

In [31]:
# predict on validation set with probabilities
y_prob = best_model.predict_proba(X_val)

# evaluate log_loss
from sklearn.metrics import log_loss
print(f'Train Log Loss: {log_loss(y_train, best_model.predict_proba(X_train))}')
print(f'Val Log Loss: {log_loss(y_val, y_prob)}')

[LightGBM] [Warning] Unknown parameter: auto_class_weights
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=7 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] feature_fraction is set=0.6440892430178381, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6440892430178381
[LightGBM] [Warning] lambda_l1 is set=3.3353038452842824e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.3353038452842824e-06
[LightGBM] [Warning] lambda_l2 is set=0.48998876620247134, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.48998876620247134
[LightGBM] [Warning] bagging_fraction is set=0.9408297820132444, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9408297820132444
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] Unknown par

### Manual Tuning Round 3

In [23]:
# manual define the params
def objective(trial):
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'boosting': trial.suggest_categorical('boosting', ['gbdt', 'dart','rf']),
        'verbosity': -1,
        'num_class': 3,
        'auto_class_weights': trial.suggest_categorical('auto_class_weights', ['balanced', 'sqrt_balanced', 'None']),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 2, 90),
        'max_depth': trial.suggest_int('max_depth', 2, 90),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 150),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': 666
        }
    train_data = Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    pruning_callback = LightGBMPruningCallback(trial, 'multi_logloss')
    cv_results = lgb.cv(
        params=params,
        train_set=train_data,
        num_boost_round = trial.suggest_int('num_boost_round', 100, 1000),
        nfold=10,
        seed=666,
        stratified=True,
        callbacks=[pruning_callback,early_stopping(10)],
        )
    return cv_results['valid multi_logloss-mean'][-1]

In [24]:
study = optuna.create_study(direction='minimize', study_name='LightGBM Manual Tuner CV', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=150)

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[10]	cv_agg's valid multi_logloss: 0.509974 + 0.0137715
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	cv_agg's valid multi_logloss: 0.491972 + 0.0136095
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[886]	cv_agg's valid multi_logloss: 0.519497 + 0.00848679
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[60]	cv_agg's valid multi_logloss: 0.492453 + 0.0123628
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[46]	cv_agg's valid multi_logloss: 0.501782 + 0.013384
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[51]	cv_

In [25]:
study.best_params

{'boosting': 'rf',
 'auto_class_weights': 'sqrt_balanced',
 'learning_rate': 0.04057984461724083,
 'num_leaves': 90,
 'max_depth': 65,
 'min_data_in_leaf': 19,
 'feature_fraction': 0.41436749321875455,
 'bagging_fraction': 0.8692968711178234,
 'bagging_freq': 3,
 'lambda_l1': 4.64286612806934e-08,
 'lambda_l2': 2.3918942324625607e-08,
 'min_child_samples': 68,
 'num_boost_round': 680}

In [26]:
# fit model with best params
best_params = study.best_params

best_model = LGBMClassifier(**best_params, random_state=666)
best_model.fit(X_train, y_train, categorical_feature=categorical_features)

[LightGBM] [Warning] Unknown parameter: auto_class_weights
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=68 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] feature_fraction is set=0.41436749321875455, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41436749321875455
[LightGBM] [Warning] lambda_l1 is set=4.64286612806934e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.64286612806934e-08
[LightGBM] [Warning] lambda_l2 is set=2.3918942324625607e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.3918942324625607e-08
[LightGBM] [Warning] bagging_fraction is set=0.8692968711178234, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8692968711178234
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] num_

LGBMClassifier(auto_class_weights='sqrt_balanced',
               bagging_fraction=0.8692968711178234, bagging_freq=3,
               boosting='rf', feature_fraction=0.41436749321875455,
               lambda_l1=4.64286612806934e-08, lambda_l2=2.3918942324625607e-08,
               learning_rate=0.04057984461724083, max_depth=65,
               min_child_samples=68, min_data_in_leaf=19, num_boost_round=680,
               num_leaves=90, random_state=666)

In [27]:
# predict on validation set with probabilities
y_prob = best_model.predict_proba(X_val)

# evaluate log_loss
from sklearn.metrics import log_loss
print(f'Train Log Loss: {log_loss(y_train, best_model.predict_proba(X_train))}')
print(f'Val Log Loss: {log_loss(y_val, y_prob)}')

[LightGBM] [Warning] Unknown parameter: auto_class_weights
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=68 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] feature_fraction is set=0.41436749321875455, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41436749321875455
[LightGBM] [Warning] lambda_l1 is set=4.64286612806934e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.64286612806934e-08
[LightGBM] [Warning] lambda_l2 is set=2.3918942324625607e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.3918942324625607e-08
[LightGBM] [Warning] bagging_fraction is set=0.8692968711178234, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8692968711178234
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] Unkn

### Manual Tuning Round 4

In [33]:
# manual define the params
def objective(trial):
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'boosting': trial.suggest_categorical('boosting', ['gbdt', 'dart','rf']),
        'verbosity': -1,
        'num_class': 3,
        'auto_class_weights': trial.suggest_categorical('auto_class_weights', ['balanced', 'sqrt_balanced', 'None']),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'max_depth': trial.suggest_int('max_depth', 2, 100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 150),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': 666
        }
    train_data = Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    pruning_callback = LightGBMPruningCallback(trial, 'multi_logloss')
    cv_results = lgb.cv(
        params=params,
        train_set=train_data,
        num_boost_round = trial.suggest_int('num_boost_round', 100, 1000),
        nfold=10,
        seed=666,
        stratified=True,
        callbacks=[pruning_callback,early_stopping(10)],
        )
    return cv_results['valid multi_logloss-mean'][-1]

In [34]:
study = optuna.create_study(direction='minimize', study_name='LightGBM Manual Tuner CV', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=150)

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[488]	cv_agg's valid multi_logloss: 0.442615 + 0.0152939
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[47]	cv_agg's valid multi_logloss: 0.503913 + 0.0146091
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[302]	cv_agg's valid multi_logloss: 0.449539 + 0.0176715
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[61]	cv_agg's valid multi_logloss: 0.513053 + 0.0137598
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[139]	cv_agg's valid multi_logloss: 0.466477 + 0.0167027
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[53]	cv_agg's valid

In [35]:
study.best_params

{'boosting': 'gbdt',
 'auto_class_weights': 'sqrt_balanced',
 'learning_rate': 0.012720978696665036,
 'num_leaves': 19,
 'max_depth': 36,
 'min_data_in_leaf': 28,
 'feature_fraction': 0.4690186771500784,
 'bagging_fraction': 0.8235664003647409,
 'bagging_freq': 3,
 'lambda_l1': 0.008968253049953313,
 'lambda_l2': 0.007039093125284371,
 'min_child_samples': 35,
 'num_boost_round': 829}

In [36]:
# fit model with best params
best_params = study.best_params

best_model = LGBMClassifier(**best_params, random_state=666)
best_model.fit(X_train, y_train, categorical_feature=categorical_features)

[LightGBM] [Warning] Unknown parameter: auto_class_weights
[LightGBM] [Warning] min_data_in_leaf is set=28, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=28
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.4690186771500784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4690186771500784
[LightGBM] [Warning] lambda_l1 is set=0.008968253049953313, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008968253049953313
[LightGBM] [Warning] lambda_l2 is set=0.007039093125284371, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007039093125284371
[LightGBM] [Warning] bagging_fraction is set=0.8235664003647409, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8235664003647409
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] num_it

LGBMClassifier(auto_class_weights='sqrt_balanced',
               bagging_fraction=0.8235664003647409, bagging_freq=3,
               boosting='gbdt', feature_fraction=0.4690186771500784,
               lambda_l1=0.008968253049953313, lambda_l2=0.007039093125284371,
               learning_rate=0.012720978696665036, max_depth=36,
               min_child_samples=35, min_data_in_leaf=28, num_boost_round=829,
               num_leaves=19, random_state=666)

In [37]:
# predict on validation set with probabilities
y_prob = best_model.predict_proba(X_val)

# evaluate log_loss
from sklearn.metrics import log_loss
print(f'Train Log Loss: {log_loss(y_train, best_model.predict_proba(X_train))}')
print(f'Val Log Loss: {log_loss(y_val, y_prob)}')

[LightGBM] [Warning] Unknown parameter: auto_class_weights
[LightGBM] [Warning] min_data_in_leaf is set=28, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=28
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.4690186771500784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4690186771500784
[LightGBM] [Warning] lambda_l1 is set=0.008968253049953313, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008968253049953313
[LightGBM] [Warning] lambda_l2 is set=0.007039093125284371, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007039093125284371
[LightGBM] [Warning] bagging_fraction is set=0.8235664003647409, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8235664003647409
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] Unknow

### Xgboost